In [36]:
%run imports.ipynb

imports loaded.


### Func def

#### pre-processing

In [27]:
print('\n> Pre-Processing func:')


> Pre-Processing func:


In [22]:
def get_striking_efficiency(name, date, data):
    
    data = data[(data.fighter==name)|(data.opponent==name)]
    data = data[data.date < date].sort_values(by='date',ascending=False).reset_index(drop=True)
    return data
    print(data[['date','fighter','f_str','f_kd','opponent','o_str','o_kd']])
    knockdowns_landed = 0
    strikes_landed = 0
    for i in range(len(data)):
        prefix = 'f_' if data.loc[i,'fighter'] == name else 'o_'
        knockdowns_landed += int(data.loc[i, prefix + 'kd'])
        strikes_landed += int(data.loc[i, prefix + 'str'])
        print(f'{name} is {prefix}: {prefix}kd={knockdowns_landed}, {prefix}str={strikes_landed}')
    
    if strikes_landed == 0:
        return 0
    return round(knockdowns_landed/strikes_landed,4)

print('    get_striking_efficiency(name, date)')

    get_striking_efficiency(name, date)


In [3]:
def get_height(item):
    
    '''return height in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return round(30.48 * float(item[0:item.find("'")]) + 2.54 * float(item[item.find("'")+1:item.find('"')]),2)

print('    get_height(item)')    

    get_height(item)


In [4]:
def get_weight(item):
    
    '''return weight in kg'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 0.45 * float(item[0:item.find('lbs.')])
    
print('    get_weight(item)')    

    get_weight(item)


In [5]:
def get_reach(item):
    
    '''return reach in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 2.54 * float(item[0:item.find('"')])
    
print('    get_reach(item)')    

    get_reach(item)


In [6]:
def get_std_reach(item, height, reach_list, height_list):
    '''return "standard reach" ( =mean(reach/height)*height )'''
    return round(height*np.mean(reach_list/height_list),2)

print('    get_std_reach(item, height, reach_list, height_list)')

    get_std_reach(item, height, reach_list, height_list)


In [7]:
def get_age(item):
    
    '''return age (instead of date of birth)'''
    
    from datetime import date,timedelta
    
    item = str(item)
    current_date = date.today().year
    
    if item[0] == '-':
        return None
    else:
        return (current_date - int(item[item.find(',')+1:]))
    
print('    get_age(item)')    

    get_age(item)


In [8]:
def get_std_age(item, age_list):
    
    '''return standrd age per weight class'''  
    return round(np.mean(age_list),2)

print('    get_std_age(item, age_list)')

    get_std_age(item, age_list)


In [9]:
def get_win_prob(name, date, df):
    
    '''return current career winning %'''
    
    wins = len(df[(df['fighter'] == name) & (df.date < date)])
    losses = len(df[(df['opponent'] == name) & (df.date < date)])
    
    if wins == 0:
        return 0
    return wins/(wins+losses)

print('    get_win_prob(name, date, df)')

    get_win_perc(name, date, df)


In [13]:
def get_win_streak(name, date, df):
    
    data = df[(df.fighter==name)|(df.opponent==name)].sort_values(by='date',ascending=False)
    data = data[data.date < date].reset_index(drop=True)
    
    if len(data) < 1:
        return 0
    
    count = 0
    for fighter in data['fighter']:
        if fighter != name:
            return count
        count += 1
    
    return count

print('    get_win_streak(name, date, df) #return count of recent wins')

    get_win_streak(name, date, df)


In [14]:
def get_lose_streak(name, date, df):
    
    data = df[(df.fighter==name)|(df.opponent==name)].sort_values(by='date',ascending=False)
    data = data[data.date < date].reset_index(drop=True)
    
    if len(data) < 1:
        return 0
    
    count = 0
    for opponent in data['opponent']:
        if opponent != name:
            return count
        count += 1
    
    return count

print('    get_lose_streak(name, date, df)')

    get_lose_streak(name, date, df)


In [4]:
def get_fighter_stats(name, data ):
    '''return fighters statistics from stat_df'''
    
    stat_list = data.columns[1:]
    row = (data[stat_list][data.name==name]).reset_index(drop=True)
    
    if len(row) < 1:
        return len(stat_list)*[None]
    else:
        return row.loc[0]
    
print('    get_fighter_stats(name, data )')    

    get_fighter_stats(name, data )


#### feature_engineering

In [28]:
print('\n> Feature Engineering func:')


> Feature Engineering func:


In [18]:
def get_stat(index, data, stat='height', fo=''):
    return data.loc[index, fo + stat]

print("    get_stat(index, data, stat='height', fo='')")

    get_stat(index, data, stat='height', fo='')


In [5]:
def get_swaped_row(index, data):
    
    swapped_row = []
    for column in data.columns:
        if column.find('f_') > -1:
            swapped_row.append(data.loc[index,'o_' + column[2:]])
        elif column.find('o_') > -1:
            swapped_row.append(data.loc[index,'f_' + column[2:]]) 
        elif column == 'result':
            swapped_row.append('lose')
        elif column == 'fighter':
            swapped_row.append(data.loc[index,'opponent'])
        elif column == 'opponent':
            swapped_row.append(data.loc[index,'fighter'])
        else:
            swapped_row.append(data.loc[index,column])    
    
    return swapped_row

print('    get_swaped_row(index, data) # return fighter & opponent statistics swapped. ')

    get_swaped_row(index, data)


In [6]:
def get_wins(combo, data):

    data = data[(data.result=='win') & (data.stance_combo==combo)]
    return len(data)

print('    get_wins(combo, data)')

    get_wins(combo, data)


In [7]:
def get_wins_ratio(combo, data):
    
    combo_df = data[data.stance_combo==combo]
    if len(combo_df) == 0:
        return 0
    else:
        return get_wins(combo, data=data)/len(combo_df)
    
print('    get_wins_ratio(combo, data)')    

    get_wins_ratio(combo, data)


#### outlier_cleaning

In [35]:
print('\n> outlier_cleaning func:')


> outlier_cleaning func:


In [42]:
def set_outlier_detection_model(X_columns, data, n=20, contemination=0.05):
    
    lof = LocalOutlierFactor(n_neighbors=n, algorithm='auto', metric='minkowski',contamination=contemination, novelty=False, n_jobs=-1)
    lof.fit(data[X_columns])
    return lof
print('    set_outlier_detection_model(X_columns, data)')

    set_outlier_detection_model(X_columns, data)


>>>>

#### model_stats_pred_training

In [8]:
print('\n> model_stats_pred_training func:')


> model_stats_pred_training func:


In [9]:
def set_lower_cases(string):
    return ''.join(c.lower() for c in string if c.isalpha())

print('    set_lower_cases(string)')

    set_lower_cases(string)


In [10]:
def set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3):

    if 'set' not in data.columns:
        data.insert(0, 'set', None) # inserd 'set' column to position 0
    
    for i in range(len(data)):
        
        if data.loc[i,'result'] == None:
            data.loc[i,'set'] = 'new'
        
        elif random.random() < test_size:
            data.loc[i,'set'] = 'test'
        else:
            if random.random() < valid_size:
                data.loc[i,'set'] = 'validation'
            else:
                data.loc[i,'set'] = 'train' 

    X_train = data[X_columns][data.set=='train']
    X_test = data[X_columns][data.set=='test']
    y_train = data[y_columns][data.set=='train']
    y_test = data[y_columns][data.set=='test']
    X_val = data[X_columns][data.set=='validation']
    y_val = data[y_columns][data.set=='validation']
    
    print('Data split: Data=%s, train_set=%s, validation_set=%s, test_set=%s'  %(str(data.shape), str(X_train.shape), str(X_val.shape), str(X_test.shape)))
    return data

print('    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)')

    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)


In [11]:
def set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5):
    
    xgboost_reg = XGBRegressor(n_estimators=n, max_depth=max_depth, learning_rate=learning_rate, subsample=subsample)
    X_train = data.loc[data.set=='train',X_columns]
    y_train = np.ravel(np.array(data.loc[data.set=='train',y_columns]).reshape(-1,1))
    xgboost_reg.fit(X_train,y_train)
    return xgboost_reg

print('    set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5)')

    set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5)


In [16]:
def set_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column + '_pred'] = None
        data[column + '_pred'] = model.predict(data[X_columns])
    
    return data

print('    set_predictions(model, data, X_columns, y_columns)')

    set_predictions(model, data, X_columns, y_columns)


In [ ]:
def get_mean_percentage_error(act, pred):
    residual = np.abs(act - pred)
    mean = np.mean(act)
    ucl = mean + 3*np.std(act)
    lcl = mean - 3*np.std(act)
    return np.mean(1 - (residual/(ucl-lcl)))

print('    get_mean_percentage_error(act, pred)')

In [ ]:
def save_trained_model(model, filename_path):
    
    with open(filename_path, 'wb') as f:
        pickle.dump(model, f)
    
    print(filename_path + ' saved.')
    
print('    save_trained_model(model, path)')    

#### result_model_training

In [ ]:
print('\n> result_model_training func:')

In [30]:
def set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs'):
    
    X_train = data[data['set'] == 'train'][X_columns]
    y_train = np.ravel(np.array(data[data['set'] == 'train'][y_columns]).reshape(-1,1))
    
    # Create a logistic regression model
    model = LogisticRegression(C=c, solver=solver, penalty=penalty)

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Create a dataframe with the predicted results
    return model

print("    set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs')")

    set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs')


In [ ]:
def get_accuracy(act, pred):
    
    true = 0
    
    for act_value, pred_value in zip(act, pred):
        if act_value == pred_value:
            true += 1
    
    return true/len(act)    
    #return accuracy_score(act, pred)

print('    get_accuracy(act, pred)')

In [ ]:
def get_mean_accuracy(act, pred):

    return np.mean([get_accuracy(act, pred),get_accuracy(act, pred),get_accuracy(act, pred)])    


print('    get_mean_accuracy(act, pred) #scoring performance x3 times and return its mean.')

In [ ]:
def get_model_evaluation(data,y_column):

    y_test_act = data[y_column][data.set=='test']
    y_test_pred = data[y_column + '_pred'][data.set=='test']

    print('Accuracy = %.6f' % accuracy_score(y_test_act,y_test_pred))
    ConfusionMatrixDisplay.from_predictions(y_test_act,y_test_pred) 
    #return accuracy_score(y_test_act,y_test_pred)
    
print('    get_model_evaluation(data)')    

In [ ]:
def load_trained_model(file_path):
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
        
    return model

print('    load_trained_model(file_path) #.PKL model from ../global_variables/')

#### method_model_training

In [37]:
print('\n> method_model_training func:')


> method_model_training func:


In [36]:
def set_knn_clf(X_columns, y_columns, data, neighbors=5):
    
    X_train = data[data['set'] == 'train'][X_columns]
    y_train = np.ravel(np.array(data[data['set'] == 'train'][y_columns]).reshape(-1,1))
    
    # Create a KNN classifier model
    model = KNeighborsClassifier(n_neighbors=neighbors)

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Create a dataframe with the predicted results
    return model

print("    set_knn_clf(X_columns, y_columns, data, neigbors=5)")

    set_knn_clf(X_columns, y_columns, data, neigbors=5)


#### predictor

In [31]:
print('\n> Predictor func:')


> Predictor func:


In [39]:
def get_current_date():
    return date.today().strftime('%Y-%m-%d')

print('    get_current_date()')

    get_current_date()


In [20]:
def get_stance_wins_ratio(fighter, stats_data):
    stance = stats_data[stats_data.name==fighter]['stance'].tolist().pop()
    data = stats_data[stats_data.stance==stance]
    wins = data.wins
    losses = data.losses
    return round(np.mean(wins/(wins + losses)),4)

print('    get_stance_wins_ratio(fighter, stats_data)')

    get_stance_wins_ratio(fighter, stats_data)
